# Modeling Neural Population with Mixture Models

## Abstract
TODO: redo all of this. Touch on:

- Why we started this. Problem with unupdated centroids.
- Wanted to incormporate homeostasis. 

The present work explores how one can model learning in neural populations as an insatnce of online Gaussian Mixture Model (GMM) learning. First we descirbe the Hard Expectation-Maximization (EM) algorithm, a common method to infere GMM's. We then show how by assuming sperichal varainces and uniform wights the Hard EM algorithm becomes one and the same with the K-means algorithm. Then we show how K-means can be seen as an istance of a Winner Takes it All (WTA) artificial neural network (ANN). 

We then briefly describe why the EM algorithm is convenient to model the system at hand and go on to relax the simplifications made in the Hard EM algorithm to give way tho the Soft EM algorithm. Finally we touch on the implications that lifting these simplifications has on the analogue nerual model. 

## WTA-ANNs. Artificial neural networks of biological interest.

Winner takes it all artificial neural networks (WTA-ANNs) have long been studied in computational neuroscience becasue they provide a simple model for a common conectivity pattern in the brain cortex, where neurons compete with each other for activation through lateral inhibition. 

WTA-ANNs are commonly 2 layered, fully conected ANNs where only one output neuron $y_i$ (we will refere to any of the $m$ neurons in the output layer as neuron $y_i$) is be activated per input. Learning takes place through Hebbian adjustments of the weights $W$ between these two layers, every time an input is presented to the network. Only the weights $\bar{w}_{:j}$ associated with the winning output neuron $y_i$ are adjusted per input, the winning neuron being the one with weights $\bar{w}_{:j}$ that are the closest, accoridng to some messure of distance, to the input vector $X_k$ (we refere to each neuron in the input layer as $x_i$, and to the whole input vector as $X_k$ where $k$ indicates the index of the input).

<img align="center" src="./img/wta.png" width="400">

Usually the goal of such a ANN is to optimally classify the inputs using the $m$ output neurons. The idea is that given an input pattern $X_k$, the asociated weights $w_{m:}$ of winning neuron $y_m$ get adjusted such that, if the same or a similar input $X_{k+1}$ is presented to the network then $y_m$ is again the winner, thereby succesfully classifying the input. 

Even thou it is conveninet to think of all $\bar{w}_{:i}$ weight associated with winning neuron $y_i$ to be updated as a whole in vector form, we can also think of the update as takeing place individually for each element of $\bar{w}_{ij}$, associated with input neuron $x_j$ and output neuron $y_i$. The weight updates then become akin to biologial Hebbain learning where learning happens locally, e.g becasue the presynaptic neuron fires shortly before the postsynaptic neuron the synaptic weight between the two is strengthened. 

If we restrict the weights and the inputs to be normalized, such that the sum of the elements in all input vectors $X_k$ and all weight vector $\bar{w}_{:i}$ are equal to a constant (TODO: tag equation), then the weight update can simply be an adjustment of the previous weight wectors $\bar{w}_{:i}$, where the adjusment is a function the difference between $\bar{w}_{:i}$ and input vector $X_k$ (TODO: tag eaquation).

$$
\sum_{w_{ij} \in \bar{w}_{:i}} w_{ij} = \alpha
$$

$$
\sum_{x_i \in X_k} x_i = \alpha
$$

$$
\bar{w}^{(t)}_{m:} \leftarrow \bar{w}^{(t-1)}_{m:} - \eta \epsilon'(X_k - \bar{w}^{(t-1)}_{m:})
$$

If we choose the squared L2 norm as our messure of distance $\epsilon(X_k - \bar{w}^{(t-1)}_{m:})$ we can take the derivative w.r.t $\bar{w}_{:i}$ and use the result as the update function $\epsilon'(X_k - \bar{w}^{(t-1)}_{m:})$. Here $\eta$ is a small positive learning rate. 

$$
\epsilon(X_i - \bar{w}^{(t-1)}_{m:})) = \frac{1}{2} ||X_k - \bar{w}^{(t-1)}_{m:})||_2^2 
$$

$$
\frac{d}{d\bar{w}^{(t-1)}_{m:}} \epsilon(X_i - \bar{w}^{(t-1)}_{m:})) =  \frac{d}{d\bar{w}^{(t-1)}_{m:}} \frac{1}{2} ||X_k - \bar{w}^{(t-1)}_{m:})||_2^2 = -(X_k - \bar{w}^{(t-1)}_{m:})) = \epsilon'(X_i - w_{m:})
$$

$$
\bar{w}^{(t)}_{m:} \leftarrow \bar{w}^{(t-1)}_{m:} + \eta (X_k - \bar{w}^{(t-1)}_{m:})
$$

The result is an online version of the K-means algoritm, where upon an inpuit the closest centroid (here the weights $\bar{w}_{m:}$), gets pulled towareds the coordiantes of the input $X_k$. A perfect equivalence with the K-means algorithm would require that all the inputs $X_k$ to be presented at once. In such case we would simply adjust our cost function to account for all $K$ data points associated to the winning neuron $y_i$. By minimizing this quantity (applying the derivative, seting it to zero and solving for $\bar{w}_{m:}$) we find that the update is the same as in the K-means algorithm, showing the equivalence between these two simingly different clustering schemas. 

$$
\epsilon(X - w_{m:}) = \frac{1}{2} \sum_{i=1}^N ||X_i - w_{m:}||_2^2 
$$

$$
\hat{w_{m:}} = \underset{w_{m:}}{\operatorname{arg min}} \frac{1}{2} \sum_{i=1}^N ||X_i - w_{m:}||_2^2 
$$

$$
\hat{w_{m:}} = \frac{\sum_i^N X_i}{N}
$$ 

TODO: insert as pseudocode, K-means algorithm

The vanilla WTA-ANN as well as the K-means algorithm have the problem that becasue only the weights $\bar{w}_{i:}$ asociated to the winnig neuron $y_i$ get updated, it is common for some centorids to be asigned two or more clusters of data points, while other centroids are left without "children" datapoints and hence never get updated. Here we call this issue uneaven membership assignmetn. This is partly due to the "hardness" of the asignment step, or in other words the fact that only one neuron can win and therefore represent a given input. By modeling centroids as being only partly responsable for a given data point, instead of belonging exlusively to one, we can somewhat bypass this problem, although not entierly since the performance of the algoruthm will sitll be highly dependent on initialization.

<img align="center" src="./img/problems_kmeans.png" width="400">

In the following section we porpose a porbabilistic interpretation of the learning procedure that is taking place in the WTA-ANNs. Then we use this proposed farmework to design a Soft-WTA where the ouputs are rather responsabilities than all-or-none responses. These repsponsabilities encode the probability that a speciofic neuron fires given a specific input pattern, and therefore allowe for multiple output neurons to get activated upon an input. 

TODO: define waether one should write $\bar{w}_{:j}$ or $\bar{w}_{i:}$ depending on the matrix W and the corresponding vector of output neuron $y_i$.

#### K-means algorithm
##### Step 0
Initialize parameters for each centroid. 

##### Step 1
$$
z_i^{(t)} = \underset{z}{\operatorname{arg min}} \| x_i - \mu_z\|_2
$$

##### Step 2
$$
\mu_z^{(t)} = \frac{\sum_i^n \mathbb{1} (z_i^{(t)} = z) x_i}{\sum_i^n \mathbb{1} (z_i^{(t)} = z)}
$$

##### Step 3
Repeat step 1 and 2 until convergence

## Mixture Models

A mixture model is a probabilistic model for representing the presence of subpopulations within an overall population, without requiring that an observed data set identifies the sub-population to which an individual observation belongs. Usually mixture models are used to make statistical inferences about the properties of the sub-populations given only observations on the pooled population (TODO: [quoute this from wikipedia](https://en.wikipedia.org/wiki/Mixture_model#:~:text=In%20statistics%2C%20a%20mixture%20model,which%20an%20individual%20observation%20belongs)).

Just by reading this description we can already sense that mixture models might be a good fit for modeling the classfication of patterns as in the WTA-ANN. Mixture models have latent and observed random variables (RVs). The observed RVs correspond in our case to the input patterns (e.g. for $\forall X_k$ neuron $X_k = [x_1, ..., x_n]$). The idea is that each obervation can be expressed as a linear combination of the latent mixture components, where the mixture weights specify the importance a component in explaining a given input. We can think of the latent variables as the identity of the causes that induce a specific input patern distributon, with the observed inputs being instances that can be expressed as a linear combination of the components. 

TODO: write equatins for the parameters.

There is two main inference problems when dealing with mixture models. On one hand one would like to know the number and functional form (parametric fmaily) of components within a mixture. This is usually referred to as the system identification proble. On the other hand one would like to estimate the corresponding parameter values given the number of components and their functional form. This is referred to as parameter estimation. 

Generally these methods consider separately the questions of system identification and parameter estimation; methods to determine the number and functional form of the components within a mixture are distinguished from methods to estimate the corresponding parameter values. In this work we concer ourselfs with he problem of parameter estimation, and we choose Expectation Maximization as or method of choice for inference (TODO: footnote explinng that the reason why this is is mainly convenience, beacue we can esaly show that a cosntrained form EM is equivlaent yo K-means).

In the context of mixture models, the EM algorithm can be seen as unsupervised learnign algorithm. This is interesting becase in general it might be useful to reason about learning in the brain as an unsupervised learnign procedure, where labels the are stimuly which generate specific input pattern distributions that the barin then categorizes as belonging to the unldeying stimuly by tuning the weights of the synapses (TODO: cite here Feedforward Inhibition and Synaptic Scaling paper). Then it is natural to think of these undelying stimuly as hidden random variables, with the observed data being the patterns that they generate.

## Gaussian Mixture Models

In the following we show that by considering the components of a mixture model to be Gaussian and by further constraining it, namely by inforcing uniform and unit varaince in all components and uniform mixture weights, we recover the K-means algorithm. We call this the Hard EM algorithm. We then show how by easing up these restictios we can get a soft membership assignemt and ease the uneaven membership problem. This we call the Soft EM. 

### The Hard EM 
The EM algorithm iterates until convergence. One iteration of the EM starts by performming an expectations step (E-step), where expectation values for the membership variables of each data point are computed. The next step is called the Maximization step (M-step) where using the previosly estimated expected memberships, the parameter for the distributioons are estimated. For the Hard EM in the classification setting the E-step classifies each data point as belonging to only one of the clusters. We denote this correspondance as $z_i$, where $i$ indicates the index of a specific data point. In the following equations superscript $(t)$ indicates the iteration in the algoritm and $\theta$ our model parameters, namely $\mu_i$ and $\Sigma_i$ the mean and covariance matrix of cluster $i$.

#### E-step
$$
z_i^{(t)} = \underset{z}{\operatorname{arg max}} P(z | x_i, \theta^{(t-1)})
$$

We can decompose $P(z | x_i, \theta^{(t-1)})$ using Bayes rule to get the follwoing expression.

$$
z_i^{(t)} = \underset{z}{\operatorname{arg max}} \frac{1}{Z} P(z | \theta^{(t-1)}) P(x_i | z, \theta^{(t-1)}) 
$$

$P(z | \theta^{(t-1)})$ is the probabity of any data point belonging to cluster $z$. By assuming that each cluster has the same size, these probabilities are the same for all clusters (e.g. $\frac{1}{K}$) and hence do not play a role in the maximization argument. $P(x_i | z, \theta^{(t-1)})$ is the probability of our data point belonging to cluster $z$ given our past estimate of the parameteres $\theta^{(t-1)}$, namely $\mathcal{N}(x_i; z, \theta^{(t-1)})$. $\frac{1}{Z}$ is the normalization constant from Bayes rule which is also independen of $z$ and therefor can be omited in the maximization. These observations give way to the folowing expression.

$$
z_i^{(t)} = \underset{z}{\operatorname{arg max}} \frac{1}{\sqrt{(2\pi\sigma^2)^d}} \exp(\frac{1}{2\sigma^2} \| x_i - \mu_z^{(t-1)}\|_2^2 )
$$

We can see again that the maximization is only over the squared 2-norm and we can aslo note that maximizing this quantity is equivalent to minimizing the 2-norm.

$$
z_i^{(t)} = \underset{z}{\operatorname{arg min}} \| x_i - \mu_z^{(t-1)} \|_2
$$

Hence the chosen cluster is the one with the minimum distance to the data point, just like in K-means and in the WTA_ANN.

#### M-step
Thanks to the estimates of the cluster membership computed in the E-step we now have a full "labeled" data set $D^{(t)} = \{ (x_1, z_1^{(t)}), \dots, (x_n, z_n^{(t)})\}$ and can now do maximum likelihood estimation (MLE) on the following quantitiy. 

$$
\theta^{(t)} = \underset{\theta}{\operatorname{arg max}} P(D^{(t)} | \theta)
$$

First we note that we can rewrite $P(D^{(t)} | \theta)$ as $P(x_{1:n}, z_{i:n} | \theta)$ and then use the chain rule to write $P(x_{1:n} | z_{i:n}, \theta) P(z_{i:n} | \theta)$. Then by applying the log we get the following expression. 

$$
LL(\theta) = \log P(x_{1:n}, z_{i:n} | \theta) = \log P(x_{1:n} | z_{i:n}, \theta) + \log P(z_{i:n} | \theta)
$$

We know from the preovious derivation that $P(z_{i:n} | \theta^{(t-1)})$ is the mixture weights $w_i$ sociated with cluster $i$ and that $P(x_{i:n} | z_{i:n}, \theta^{(t-1)})$ are normal normal distributions with parameters $\theta^{(t-1)}$. Then by assuimng that the data points are conditionally independent we arrive at the following expression.

$$
= \sum_i^n \log w_i - \log \frac{1}{\sqrt{(2\pi\sigma^2)^d}} - \frac{1}{2\sigma^2} || x_i - \mu_z ||_2^2 
$$

Finally we can apply perform MLE on this expression by applying the derivative and setting it to zero to find the expression for the update of our parameters. Since in this specific case we are only updating the means $\mu_z$ and leaving both the weights and the variances untoched we only show the expresion for the fromer. 

$$
\mu_z^{(t)} = \frac{\sum_i^n \mathbb{1} (z_i^{(t)} = z) x_i}{\sum_i^n \mathbb{1} (z_i^{(t)} = z)}
$$

We can now see that by considering fixed spherical unit variances and weights the Hard EM perfomres exactly the same operations as the K-means algorithm.

### The Soft EM
Techincially speaking one could also perform the Hard EM algorithm relaxing both the unit spherical covariance and fixed uniform weights asusmtion. This is equivalent to allowing the underlying cluster to have different sizes and eliptical shapes, which might already help further dilute uneaven membership assignment. However, the Hard EM will greedly assign a data point to one of the cluster even thou the probability that it belongs to either of them is the same or almost the same. This may lead to poor performance specially when the clusters are overlaping. By using membership intead of hard assignments we account for this uncertainty and also help poorly intialized clusters to get updated, since even if they are far away from data points, will still have a responsability over them.

#### E-step
Now we seek to find the responsability $\gamma_j(x_i)$ of each cluster on each data point $x_i$, or in other words the probability that data point $x_i$ belongs to cluster $j$. Here $\theta$ encompases $\Sigma, \mu$ and $w$.

$$
\gamma_j(x_i) = P (z = j | x_i, \theta) = \frac{P(z = j, x, \theta)}{P(x, \theta)} = \frac{P(x|z = j, \theta) P(z = j | \theta)}{P(x| \theta)}
$$

$$
P(x|\theta) = \sum_k P(x|\theta, z = k) P(z = k|\theta)
$$

We know from before that $P(z = j|\theta) = w_j$, and that $P(x|\theta, z = j) = \mathcal{N}(x; \mu_j, \Sigma_j)$, hence we can write the following.

$$
\gamma_j(x_i) = \frac{w_j\mathcal{N}(x; \mu_j, \Sigma_j)}{\sum_k w_k \mathcal{N}(x;\mu_k, \Sigma_k)}
$$

Once we have all the $\gamma$'s for each data point, we again have a fully annotated data set, namely labels, which are now the responsabilities, and the cordinates of the data points themselves. We can now move one to infere the parametrs $\theta$ in the M-step.

#### M-step
By aplying MLE on $\log P(x_{1:n}, z_{i:n} | \theta)$ we can get closed form expression for the parameters $w_j, \mu_j$ and $\Sigma_j$, for every cluster $j$ (derivation not shown).

$$
w_j^{(t)} \leftarrow \frac{1}{n} \sum_{i=1}^n \gamma_j^{(t)}(x_i) 
$$

$$
\mu_j^{(t)} \leftarrow \frac{\sum_{i=1}^n \gamma_j^{(t)}(x_i)x_i}{\sum_{i=1}^n \gamma_j^{(t)}(x_i)}
$$

$$
\Sigma_j^{(t)} \leftarrow \frac{\sum_{i=1}^n \gamma_j^{(t)}(x_i)(x_i-\mu_j^{(t)})(x_i-\mu_j^{(t)})^T}{\sum_{i=1}^n \gamma_j^{(t)}(x_i)}
$$

AQUI_NOS_QUEDAMOS

TODO: maybe now we want go into why the normal is not the best, since we are deling with inputs that shoudl represent fiering rates. Maybe we want to avoid about neurons being not neurons but populations since this would make us loose analogy with a biologicla network, oder?

## Winner Takes it All

We have shown that K-means is equivalent to a resticted version of the Hard EM algorithm. Now we show that it is also equivalent to a WTA network. Given a 2 layered, fully conected artificial neural network (ANN), a WTA network is one where given an input, only one neuron $y_i$ in the output layer will be activated.

<img align="center" src="./img/wta.png" width="400">

Then, given a series of inputs $X_i = (x_1, ..., x_n)$, we seek optimally classify the inputs using our $m$ output neurons. We train our network by adjusting the $w_{nm}$ weights in the network. The idea is that given an input pattern $X_i$, the  asociated weights $w_{m:}$ of winning neuron $y_m$ get adjusted such that, if the same or a similar input $X_i'$ is presented to the network then $y_m$ is again the winner, thereby succesfully classifying the input. Following this rationale it is only natural to adjust the weights $w_{m:}$ by performing gradient decent w.r.t some error that is a function of the difference between the weights $w_{m:} \in \mathbb{R}^n$ and $X_i \in \mathbb{R}^n$. Note that this imposes some restirction on the values that the $w_{mn}$'s can take, namely they must be in the same order of magnitude as the inputs $X_i$ themselves. To ensure this one can simply normalize both the weights, the outputs and the inputs.

$$
w_{m:} \leftarrow w_{m:} - \epsilon'(X_i - w_{m:})
$$

If we choose the squared L2 norm as our messure of distance we can take the derivative w.r.t $w_{m:}$ and use the result as the update function $\epsilon'(X_i - w_{m:})$. One can also modify the learinign schema such that a learning rate $\eta$ is incorporated as $\eta \epsilon'(X_i - w_{m:})$ to control the magnitude of the adjustents, however here we stick to the vanilla version ommiting such term. 

$$
\epsilon(X_i - w_{m:}) = \frac{1}{2} ||X_i - w_{m:}||_2^2 
$$

$$
\frac{d}{dw_{m:}} \epsilon(X_i - w_{m:}) =  \frac{d}{dw_{m:}} \frac{1}{2} ||X_i - w_{m:}||_2^2 = -(X_i - w_{m:}) = \epsilon'(X_i - w_{m:})
$$

$$
w_{m:} \leftarrow w_{m:} + (X_i - w_{m:})
$$

The result is an online version of the K-means algoritm, where upon an inpuit the closest centroid (here the weights $w_{m:}$), gets pulled towareds the cordiantes of the input data. A perfect equivalence with the K-means algorithm would require that all the inputs $X_i$ to be presented at once. In such case we would simply adjust our cost function to account for all $N$ data points associated to the output $y_m$. By minimizing this quantity (applying the derivative, seting it to zero and solving for $w_{m:}$) we find that the update is the same as in the K-means algoritm, showing the equivalence between these two simingly different clustering schemas. 

$$
\epsilon(X - w_{m:}) = \frac{1}{2} \sum_{i=1}^N ||X_i - w_{m:}||_2^2 
$$

$$
\hat{w_{m:}} = \underset{w_{m:}}{\operatorname{arg min}} \frac{1}{2} \sum_{i=1}^N ||X_i - w_{m:}||_2^2 
$$

$$
\hat{w_{m:}} = \frac{\sum_i^N X_i}{N}
$$

### Why are WTA networks of biological interest?
In a nutshell, becasue they provide a simple model for a common conectivity pattern in the brain cortex, where neurons compete with each other for activation through lateral inhibition. 

We can desgin an biological circuit of toy neurons analogus to a WTA ANN, where neurons change their synaptic weights according to Hebbian learing. Furthermore, by discretising the activation of the output neurons in this toy biological circuit, such thay only one neuron gets activated given an input, one can recover the previously described WTA ANN, and thereby an online version of the K-means algorithm. The winning neuron will therefore be the only one to adjuste the weights of the corresponding synapses, thereby makeing it more likely that upon a similar input the same output neuron will fire. It is important to note that in such a biological system the update of the weight happens locally, in the sens that if neuron $y_m$ is activated the weight $w_{nm}$ will be strengthened through Hebbian learining if there was an input from neuron $n$ (e.g the presynaptic neuron fires shortly before the postsynaptic neuron). Even thou previosuly we considered all $w_{:m}$ weight associated with neuron $y_m$ to be updated as a whole in a vector form, we can also think of the update takeing place individually for each element of $w_{n:m}$, asciated with input neuron $x_n$ and output neuron $y_m$.

A more realistic model would allow for the posibility that multiple output neurons get activated given an input, even when the system is tuned such that one output neuron almost always gets acitaved when recivering a specific input parrtern while the other output neurons seldomly react to such input. Here we propose a Soft-WTA implmentation thorough responsabilities which encode the probability that a speciofic neuron fires given a specific input pattern. 

### Towards a Soft-WTA
Standard WTA updates the weighst of only one of the output neurons, the one with the closes Eucledian distance from the input. Here we aim to update every neuron accroding to the responsabiliy they have over a given data point. Specifically we weight the distance from a data point $X_i$ to the weights $w_{m:}$ asociated with cluster $m$ by the responsability $\gamma_m(X_i)$ of clsuter $m$ over data point $X_i$.

$$
\epsilon(X_i - w_{m:}) = \frac{\gamma_m(X_i)}{2} ||X_i - w_{m:}||_2^2 
$$

In the batch version of WTA we would have to account for all $N$ data points that are presented to the network and summ the distances weighted by the correspondign responsabilities.

$$
\epsilon(X - w_{m:}) = \frac{1}{2} \sum_{i=1}^N \gamma_m(X_i) ||X_i - w_{m:}||_2^2 
$$

By applying the derivative, seting it to zero and solving for $w_{m:}$ we get the following.

$$
\epsilon'(X - w_{m:}) = - \sum_{i=1}^N \gamma_m(X_i) (X_i - w_{m:})
$$

$$
0 = \sum_{i=1}^N \gamma_m(X_i) w_{m:} - \sum_{i=1}^N \gamma_m(X_i) X_i
$$

$$
w_{m:} = \frac{\sum_{i=1}^N \gamma_m(X_i) X_i}{\sum_{i=1}^N \gamma_m(X_i)}
$$

We can see that weighting the the error by the responsability $\gamma_m(X_i)$ of cluster $m$ over datapoint $X_i$, in the bach scenario recovers the Soft EM update for the mean of the clusters (here $w_:m$). In order to obtain the online version of the Soft-WTA, we follow the same strategy as before, namely we take the old estimate for the weights and we take a step in the oposite direction of the gradient of the error $\epsilon(X_i - w_{m:})$.

$$
w_{m:} \leftarrow w_{m:} - \epsilon'(X_i - w_{m:})
$$

$$
\epsilon'(X_i - w_{m:}) = -\gamma_m(X_i) (X_i - w_{m:}) 
$$

$$
w_{m:} \leftarrow w_{m:} + \gamma_m(X_i) (X_i - w_{m:}) 
$$

How can this responsability be computed locally in our ANN model? We can view it as the normalized output of neuron $y_m$. By nature the normalization opertaion necesitates a division which takes as input the other non-nromalized activations of the other neruons. Naturally this is where the lateral inhibiton of the ouput neurons comes in, such that the overall activity is a function to the activity of all of the nuerons in the output layer. This division operation could be implemented with shunting inhibition of example, where the other neurons in the output layer set the conducntance such that the gain and the sumation dynamics of nenuon $y_m$ are is determined by the activity of all other neurons. This is due to the fact that the volatege of a postsynaptioc neuron can be modeled as being inversly porportional to the conductance (set by lateral shuntting inhibition) and directly proportional to the driving current resulting from the temporal and saptial sumation of inhibitory and exitatory inputs comming from the input layer. 

$$
V \propto \frac{I_d}{g}
$$

$$
I_d = g_eV_e + g_iV_i + g_{shunt}V_{shunt} + g_{leak}V_{leak}
$$

$$
g = g_e + g_i + g_{shunt} + g_{leak}
$$

Here $V_e$, $V_i$, and $V_{shunt}$ are excitatory, inhibitory, and shunt equilibrium potentials, respectively, $g_e$, $g_i$, and $g_{shunt}$ are the variable conductance resistors, and $g_{leak}$ and $V_{leak}$ determine the leak current. $I_d$ is the cell's driving current and depends on the cell's synaptic inputs but is independent of the cell's membrane potential $V$.

The authors of this poporition suggest that becasue $g_0 = g_e + g_i + g_{leak}$ is contant and therefore changes in $g$ are only due to effects from the shunting inhibition, and conversly that $V_{shunt} = 0$ such that the diriving current only depends on the changes in $g_e$ and $g_i$. The assumtions might not hold generally in which case the overall operation might only approximate division but we just wanted to point out that such division operations are possible with well known mechanisms in biological neurons (cite_here). 

## The weadding: WTA ANN's and MM's

The EM algorithm has some very nice thoretical properties, namely that the algorithm monotonically increases the likelihood and in the context of GMM's it is garanteed to converge to a local maximum, although the quality fo the solution is still higly dependent on itiailization. Neverthels GMMs have achived some remarkable succeses. The idea is that by showing that a WTA is equivalent to performing the EM algorithm we also show that our solution shares the same thoretical assurances. 

But perhaps even more importantly, it gives us a probabilistic framework which we can use to reason about the lerning proces in biological neural networks, namely the likelihood and prior which we seek to optimize as part of the posterior distribution. As we saw eralier making a choice for the likelihood asusumes that the data has a certain structure. Here we chose to work with Gauusina distributions becasue of the already studied proximity of a WTA network to the K-means algorithm, but in principal there is no reason why one could not decide in favor of a diferent types of distrubution (for example author_name decided for an exponential distribution instead cite_here). 

Likewise the prior lets us reason about the overall poribability of a data point belonging to any given cluster. Here we propose that such probability can be seen as the homeostatic neuronal exitability of the output neurons, e.g. the probability that a neuron will get activated upon an input. While our model does not learn this parameters as it is unclear how they could be encoded into the model, one can set them a priori as hyper parameters, and see how they affect the learning. 

In general it might be useful to reason about learning in the brain as an unsupervised learnign procedure, where labels are stimuly which generate specific input pattern distributions that the barin then categorizes as belonging to the unldeying stimuly by tuning the weights of the synapses (cite_here). Then it is natural to think of these undelying stimuly as hidden random variables, with the observed data being the patterns that they generate. Thinking about the system at hand in this temrs make the EM algorithm natural tool to perofrm inference.

## Simulations

TODO later:
- Explore why Gaussians are a convininet ditribution to chooose ina our mixture models
    - some ideas, cause of tuning curvs
    - caveat, posibly some negative value which is not ok since fiering rate must be positive
- adjust introduction
- ok, maybe restructure everything to simply include the probabilistic model (with the exponential distribution) and then introduce the WTA model that will somoehow be analogous to it. 

Note on Neuromorphic circuits:

* Analog subthershold circuits are used becasue they emmulate the diffusion of ions through the membrane in a neuron. This circuits are very noisy since a small change in the input wil gratly affect the output, which emulates the stochasticity in the brian.
* Open question does the sigmoid can someohow approximate the integration of the both alteral inhibitoon from the output layer and also the direct exitation from the input layer, such that the averall outcome the activation of the nueron and such activation is the one mediating the hebian leraning?

Literature:

+ [Feedforward Inhibition and Synaptic Scaling – Two Sides of the Same Coin?](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3310709/#pcbi.1002432-Yuille1)
+ [Normalization as a canonical neural computation](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3273486/#R45)
+ [Summation and Division by Neurons in Primate Visual Cortex](https://www.jstor.org/stable/2884085)
+ [Homeostatic Regulation of Neuronal Excitability](http://www.scholarpedia.org/article/Homeostatic_Regulation_of_Neuronal_Excitability)
+ [Input normalization by global feedforward inhibition expands cortical dynamic range](https://www.nature.com/articles/nn.2441)
+ [Lognormal firing rate distribution reveals prominent fluctuation–driven regime in spinal motor networks](https://elifesciences.org/articles/18805#abstract)

Questions:

+ How do neurons encode the covariance of the inoputs? Covariance is, given a component of the mixture, the information of how one of the coordinates in the random vecotr varies when as anothe relement vaires. Given that the synaptic weights are the menas of the Gaussian components, how are the covariances of these components encoded and updated?
+ Is it ok to leave the weights (size of the components) as a hyper parameter set by the nature of the neuron population? Should individual neurons change this? How? Maybe with some kind of running averge.